In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import fitz
from fitz import Rect
import sys, re

from pdf_scraper.doc_utils   import open_exam, get_doc_line_df, get_images,preproc_images
from pdf_scraper.doc_utils   import identify_all_page_clusters, enrich_doc_df_with_images
from pdf_scraper.line_utils  import clean_line_df

from pdf_scraper.image_utils import get_bboxed_page_image 
from pdf_scraper.line_utils import get_category_boxes

The purpose of this notebook is to identify good clusterings and then choose these page/year comboes to be used in the tests.

(2001, 2), (2001, 3), (2001, 4), (2001, 6)

2001, 3 is good because this test will fail if we ever decide to join up lines which have been split horizontally.
2001, 6 is an example of title and subtitle being clumped together, which is not a problem really, but it is not desirable either.
- this would be an example of a problem to be solved by post clustering category-based separation.

In [ ]:
paper=1
level="al"
subject="english"

year = 2011

doc = open_exam(year, subject, level,paper)
df = get_doc_line_df(doc)

doc_width     = doc[0].rect.width

images = get_images(doc)
images = preproc_images(images)

df = clean_line_df(df)
df = enrich_doc_df_with_images(df,images)
df = identify_all_page_clusters(df,2.0/3.0, 1.15, True)

In [ ]:
page = 6
page_df = df[df.page==page]

rects = get_category_boxes(page_df, "cluster")
labels = np.unique(page_df.cluster )
img = get_bboxed_page_image(doc,page,rects, labels=labels )
display(img)

In [ ]:
test_df = page_df[["text","cluster"]].sort_values(by="cluster")
ref_path = Path.cwd() / "resources" / "expected_clusters" / f"{subject}_{level}_{paper}_{year}_{page}.parquet"
test_df.to_csv(ref_path)
ref_df = pd.read_csv(ref_path,index_col=0)
assert (test_df == ref_df).all().all()
(test_df == ref_df).all().all()